In [47]:
import random,subprocess
from enum import Enum
from collections import Counter

    
def generate_model_line(s,idx):
    feature_list=[]
    raw_sentence=s.split('\t')[0]
    sentence=[x for x in list(''.join(set(''.join(raw_sentence)))) if x not in insignificant_tokens]
    model_line =str(idx+1)
    #add unigrams
    for ch in sentence:
        feature_list.append(character_numbers[ch])
        
    #add bigrams
    bigrams=set()
    for words in raw_sentence.split(' '):
        bigrams.update([x.lower() for x in[words[i:i+2] for i in range(len(words)-1)]])
    bigrams_list=list([x for x in bigrams if (x[0] not in insignificant_tokens)and(x[1] not in insignificant_tokens)])
    for bg in bigrams_list:
        feature_list.append(character_numbers[bg])
    
    feature_list=list(set(feature_list))
    feature_list.sort()
    for ch in feature_list:
        model_line+=" "+ str(ch)+ ":" + str(1)
    return model_line+"\n"

insignificant_tokens=[' ','!', '"','#','$','%','&','*','+','-','(',')',',','.','/','0','1','2','3','4','5','6','7','8',
                      '9',';','<','>','=','?','@','|','«','»','`','[',']',"'",'\\']
language_ids=['bg','bs','cz','es-AR','es-ES','hr','id','mk','my','pt-BR','pt-PT','sk','sr']
with open ("Corpus/Raw Corpus.txt") as f:
    corpus = f.readlines()
languages=[]
corpus_s=list(''.join(set(''.join(corpus))))
corpus_s=[x for x in corpus_s if x not in insignificant_tokens]
character_numbers={}
i=0
for s in corpus_s:
    i+=1
    character_numbers[s]=i

bigrams=set()
for s in corpus:
    for words in s.split(' '):
        bigrams.update([x.lower() for x in[words[i:i+2] for i in range(len(words)-1)]])
bigrams_list=list([x for x in bigrams if (x[0] not in insignificant_tokens)and(x[1] not in insignificant_tokens)])
for bg in bigrams_list:
    i+=1
    character_numbers[bg]=i
for i in range(13): #divides data set as languages
    languages.append("")
    start=i*2000
    end=(i+1)*2000
    languages[i]=corpus[start:end] #2k sentence for each language
training_set=[]
test_set=[]
training_model=[]
test_model=[]
for idx,l in enumerate(languages):
    random.shuffle(l)
    
    training_partition=l[0:1800]
    training_set.extend(training_partition)
    
    for s in training_partition:
        training_model.append(generate_model_line(s,idx))
    
    test_partition=l[1800:2000]
    test_set.extend(test_partition)
    
    for s in test_partition:
        test_model.append(generate_model_line(s,idx))

In [48]:
def write_files(file_name,array):
    with open (file_name, mode='wt') as t_file:
        for item in array:
            t_file.write(item)
print(len(training_set))
print(len(test_set))
write_files("SVM/IncreasedFeatures/TrainingModel-SVM.txt",training_model)
write_files("SVM/IncreasedFeatures/TrainingSet-SVM.txt",training_set)
write_files("SVM/IncreasedFeatures/TestSet-SVM.txt",test_set)
write_files("SVM/IncreasedFeatures/TestModel-SVM.txt",test_model)

23400
2600


In [49]:
p=subprocess.Popen(['SVM/svm_multiclass_learn', '-c', '5000' ,'SVM/IncreasedFeatures/TrainingModel-SVM.txt' ,'SVM/IncreasedFeatures/Model'],
                 stdout=subprocess.PIPE)
#for line in p.stdout:
#    print(line)
p.wait()



0

In [50]:
p=subprocess.Popen(['SVM/svm_multiclass_classify', 'SVM/IncreasedFeatures/TestModel-SVM.txt' ,'SVM/IncreasedFeatures/Model'],
                 stdout=subprocess.PIPE)
for line in p.stdout:
    print(line)
p.wait()


Reading model...done.

Reading test examples... (2600 examples) done.

Classifying test examples...done

Runtime (without IO) in cpu-seconds: 0.03

Average loss on test set: 20.3462

Zero/one-error on test set: 20.35% (2071 correct, 529 incorrect, 2600 total)



0